### Análisis de Horas Críticas del Mercado para el Balance de Potencia ###
**Author:** Aminadab Cordova

**Date:** 2024-05-02

In [1]:
# Módulos requeridos
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Conexión a Base de Datos

In [3]:
# Consulta SQL para las horas criticas del año de producción
sql_query = """SELECT * FROM public."Horas_Criticas_AP" """

# Ejecutar la consulta y cargar los resultados en un DataFrame
df_hc_ap = pd.read_sql(sql_query, cnx_mbp)

In [120]:
# Se crea el dataframe con horas criticas definitivas del SIN
df_hc_ap_sin = df_hc_ap[df_hc_ap["Zona_Potencia"] == "SIN"]
df_hc_ap_sin = df_hc_ap_sin[df_hc_ap_sin["Tipo_Dato"] == "Definitivo"]

# se forma dataframe solo con fecha y hora
df_hc = df_hc_ap_sin[['Fecha', 'Hora']].reset_index(drop=True)
df_hc['Fecha'] = pd.to_datetime(df_hc['Fecha'])

# Convertir la columna 'Hora' a formato int64
df_hc['Hora'] = df_hc['Hora'].astype('int64')


In [134]:
# Se define el rango de fechas de los años de Produccion
date_range = pd.date_range('2016-01-01', '2023-12-31')

# Se repite cada día para formar una serie horaria
fecha = np.repeat(date_range, 24)

# se crea dataframe conla serie de fechas
df_serie = pd.DataFrame (fecha, columns=['Fecha'])

# Se define un arreglo de las horas y se agrega a df_serie
hour=np.arange(start=0, stop=24, step=1)
# Convertir a int64
hour = hour.astype(np.int64)


df_serie['Hora'] = np.tile(hour, len(date_range))

df_serie

,Fecha,Hora
0,2016-01-01,0
1,2016-01-01,1
2,2016-01-01,2
3,2016-01-01,3
4,2016-01-01,4
...,...,...
70123,2023-12-31,19
70124,2023-12-31,20
70125,2023-12-31,21
70126,2023-12-31,22


In [52]:
# Realizar una combinación externa (outer join) en los dos DataFrames
df_merged = df_serie.merge(df_hc, on=['Fecha', 'Hora'], how='outer', indicator=True)

# Agregar una columna llamada 'HC' que indique si la hora es crítica o no
df_merged['HC'] = df_merged['_merge'] == 'both'

# Eliminar la columna de indicador '_merge'
df_merged = df_merged.drop('_merge', axis=1)

In [54]:
df_merged[df_merged['HC'] == True]

,Fecha,Hora,HC
1623,2016-03-08,16,True
1624,2016-03-08,17,True
2367,2016-04-08,16,True
2368,2016-04-08,17,True
2369,2016-04-08,18,True
...,...,...,...
67867,2023-09-28,20,True
67868,2023-09-28,21,True
67869,2023-09-28,22,True
68034,2023-10-05,19,True


In [56]:
df_hc

,Fecha,Hora
0,2016-05-23,16
1,2016-05-23,17
2,2016-05-24,15
3,2016-05-24,16
4,2016-05-24,17
...,...,...
795,2023-09-28,20
796,2023-09-28,21
797,2023-09-28,22
798,2023-10-05,19


In [57]:
df_merged = df_serie.merge(df_hc, on=['Fecha', 'Hora'], how='inner', indicator=True)

In [58]:
df_merged

,Fecha,Hora,_merge
0,2016-03-08,16,both
1,2016-03-08,17,both
2,2016-04-08,16,both
3,2016-04-08,17,both
4,2016-04-08,18,both
...,...,...,...
795,2023-09-28,20,both
796,2023-09-28,21,both
797,2023-09-28,22,both
798,2023-10-05,19,both


In [124]:
# Convertir la columna 'Hora' a formato de cadena de texto
df_hc['Hora'] = df_hc['Hora'].astype(str)

# Restar 1 a todas las horas
df_hc['Hora_offset'] = (df_hc['Hora'].astype(int) - 1) % 24

# Convertir la columna 'Hora' a formato de cadena de texto
df_hc['Hora_offset'] = df_hc['Hora_offset'].astype(str)

# Combinar la columna 'Fecha' y 'Hora' en una nueva columna 'Fecha_Hora'
df_hc['Fecha_Hora'] = df_hc['Fecha'].dt.strftime('%Y-%m-%d') + ' ' + df_hc['Hora_offset'] + ':00'

# Convertir la columna combinada 'Fecha_Hora' a un objeto de fecha y hora
df_hc['Fecha_Hora'] = pd.to_datetime(df_hc['Fecha_Hora'], format='%Y-%m-%d %H:%M')

# Suponiendo que tienes una columna llamada 'Hora_offset' que deseas eliminar
df_hc = df_hc.drop(columns=['Hora_offset'])



In [125]:
df_hc

,Fecha,Hora,Fecha_Hora
0,2016-05-23,16,2016-05-23 15:00:00
1,2016-05-23,17,2016-05-23 16:00:00
2,2016-05-24,15,2016-05-24 14:00:00
3,2016-05-24,16,2016-05-24 15:00:00
4,2016-05-24,17,2016-05-24 16:00:00
...,...,...,...
795,2023-09-28,20,2023-09-28 19:00:00
796,2023-09-28,21,2023-09-28 20:00:00
797,2023-09-28,22,2023-09-28 21:00:00
798,2023-10-05,19,2023-10-05 18:00:00


In [127]:
df_serie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70128 entries, 0 to 70127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Fecha   70128 non-null  datetime64[ns]
 1   Hora    70128 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.1 MB


In [128]:
df_serie

,Fecha,Hora
0,2016-01-01,1
1,2016-01-01,2
2,2016-01-01,3
3,2016-01-01,4
4,2016-01-01,5
...,...,...
70123,2023-12-31,20
70124,2023-12-31,21
70125,2023-12-31,22
70126,2023-12-31,23


In [135]:
# Convertir la columna 'Hora' de df_serie a formato de cadena de texto
df_serie['Hora'] = df_serie['Hora'].astype(str)

# Crear la columna 'Fecha_Hora' en df_serie combinando la columna 'Fecha' con la columna 'Hora' ajustada
df_serie['Fecha_Hora'] = df_serie['Fecha'].astype(str) + ' ' + df_serie['Hora'] + ':00'

# Convertir la columna 'Fecha_Hora' a tipo datetime
df_serie['Fecha_Hora'] = pd.to_datetime(df_serie['Fecha_Hora'], format='%Y-%m-%d %H:%M')

# Ahora, df_serie contiene la columna 'Fecha_Hora' con las fechas y horas combinadas



In [136]:
df_serie

,Fecha,Hora,Fecha_Hora
0,2016-01-01,0,2016-01-01 00:00:00
1,2016-01-01,1,2016-01-01 01:00:00
2,2016-01-01,2,2016-01-01 02:00:00
3,2016-01-01,3,2016-01-01 03:00:00
4,2016-01-01,4,2016-01-01 04:00:00
...,...,...,...
70123,2023-12-31,19,2023-12-31 19:00:00
70124,2023-12-31,20,2023-12-31 20:00:00
70125,2023-12-31,21,2023-12-31 21:00:00
70126,2023-12-31,22,2023-12-31 22:00:00


In [137]:
df_hc

,Fecha,Hora,Fecha_Hora
0,2016-05-23,16,2016-05-23 15:00:00
1,2016-05-23,17,2016-05-23 16:00:00
2,2016-05-24,15,2016-05-24 14:00:00
3,2016-05-24,16,2016-05-24 15:00:00
4,2016-05-24,17,2016-05-24 16:00:00
...,...,...,...
795,2023-09-28,20,2023-09-28 19:00:00
796,2023-09-28,21,2023-09-28 20:00:00
797,2023-09-28,22,2023-09-28 21:00:00
798,2023-10-05,19,2023-10-05 18:00:00


In [138]:
# Realizar una combinación externa (outer join) en los dos DataFrames
df_merged = df_serie.merge(df_hc, on=['Fecha_Hora'], how='outer', indicator = True)

# Agregar una columna llamada 'HC' que indique si la hora es crítica o no
df_merged['HC'] = df_merged['_merge'] == 'both'

# Eliminar la columna de indicador '_merge'
df_merged = df_merged.drop('_merge', axis=1)

In [140]:
df_merged[df_merged['HC'] == True]

,Fecha_x,Hora_x,Fecha_Hora,Fecha_y,Hora_y,HC
1623,2016-03-08,15,2016-03-08 15:00:00,2016-03-08,16,True
1624,2016-03-08,16,2016-03-08 16:00:00,2016-03-08,17,True
2367,2016-04-08,15,2016-04-08 15:00:00,2016-04-08,16,True
2368,2016-04-08,16,2016-04-08 16:00:00,2016-04-08,17,True
2369,2016-04-08,17,2016-04-08 17:00:00,2016-04-08,18,True
...,...,...,...,...,...,...
67867,2023-09-28,19,2023-09-28 19:00:00,2023-09-28,20,True
67868,2023-09-28,20,2023-09-28 20:00:00,2023-09-28,21,True
67869,2023-09-28,21,2023-09-28 21:00:00,2023-09-28,22,True
68034,2023-10-05,18,2023-10-05 18:00:00,2023-10-05,19,True
